# Evolver Loop 2 Analysis: Understanding B3 Failure & Next Steps

## Goal
Analyze why EfficientNet-B3 with enhanced augmentation performed worse than B0 baseline, and identify high-ROI improvements.

## Key Questions
1. What caused the performance degradation in exp_003?
2. How can we build on the successful B0 baseline?
3. What ensemble strategies will give best ROI?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Load session state to review experiments
with open('/home/code/session_state.json', 'r') as f:
    session_state = json.load(f)

experiments = session_state['experiments']
print("Experiment Summary:")
print("=" * 50)
for exp in experiments:
    print(f"{exp['name']}: {exp['score']:.6f} (model: {exp['model_type']})")
    print(f"  Notes: {exp['notes'][:200]}...")
    print()

print(f"\nGold threshold: 0.038820")
print(f"Best score: {min([exp['score'] for exp in experiments]):.6f}")
print(f"Improvement needed: {0.038820 - min([exp['score'] for exp in experiments]):.6f}")